In [16]:
%run "00_setup_paths_and_imports.py"
from model_core import pot_density, k_W14_from_U2, K0_weiss74_CO2, subset_box_0360

# -------------------------------------------------------------------
# Choices
# -------------------------------------------------------------------
region_name = "coast"  # "mike_station" or "coast"
mld_mode    = "static"        # "static" or "variable" or "photic"
oae_mode    = "ctrl"           # "oae" or "ctrl"

# -------------------------------------------------------------------
# Load configuration and layer geometry
# -------------------------------------------------------------------
config = np.load(DATA_INTERIM / f"config_{region_name}.npy", allow_pickle=True).item()

region = config["region"]
lat_min = region["lat_min"]
lat_max = region["lat_max"]
lon_min = region["lon_min"]
lon_max = region["lon_max"]
area_m2 = region["area_m2"] 

times   = config["times"]    # length nt+1
centers = config["centers"]  # length nt
nt      = config["nt"]
dt      = config["dt"]

params        = config["params"]
initial_state = config["initial_state"]

# load MLD / layer geometry
layers   = np.load(DATA_INTERIM / f"layers_{mld_mode}_{region_name}.npz")
h1       = layers["h1"]      # array, length nt
h2       = layers["h2"]
H        = layers["H"]       # scalar
delta_z  = layers["delta_z"] # array, length nt

# representative lon/lat for density
lon0 = 0.5 * (lon_min + lon_max)
lat0 = 0.5 * (lat_min + lat_max)

KD_m2_yr = KD * SECONDS_PER_YEAR  # m^2/yr
pCO2_atm = 420  # µatm

# -------------------------------------------------------------------
# Load forcings: U2 and NCP
# -------------------------------------------------------------------
U2_for_step = np.load(DATA_INTERIM / f"U2_{region_name}.npy")
NCP_for_step = np.load(DATA_INTERIM / f"NCP_{mld_mode}_{region_name}.npy")


# OAE

In [17]:
# -------------------------------------------------------------------
# OAE (lime addition) setup
# -------------------------------------------------------------------
mass_lime_tonnes = 742_500.0                 # metric tonnes of Ca(OH)2
mass_lime_g = mass_lime_tonnes * 1e6     # g

# --- Ca(OH)2 molar mass calculation ---
# Ca(OH)2 = 1 Ca + 2 O + 2 H in g/mol
# Convert to kg per mol
M_CaOH2_g_per_mol = (M_Ca + 2*(M_O + M_H))
n_lime_mol = mass_lime_g / M_CaOH2_g_per_mol  

# Shock timing and surface box volume at shock time
shock_time = 2.0  # years (model time where OAE is applied)
shock_step_idx = int(np.argmin(np.abs(times[1:] - shock_time)))
h1_shock = float(h1[shock_step_idx])

# Mass of surface box water (kg)
M_water_surface_kg = rho  * area_m2 * h1_shock # h1 at shock time!
M_water_surface_Gt = M_water_surface_kg / 1e12  # in gigatonnes

# ΔTA1 in µmol/kg from complete lime dissolution, no CaCO3 precipitation, 1 mol Ca(OH)2 → 2 mol TA
delta_TA1_umol_per_kg = (2.0 * n_lime_mol / M_water_surface_kg) * 1e6

print(f"Lime addition: {mass_lime_tonnes:,.0f} tonnes Ca(OH)2")
print(f"molar mass Ca(OH)2: {M_CaOH2_g_per_mol:.4f} g/mol")
print(n_lime_mol)
print(f"→ ΔTA1 ≈ {delta_TA1_umol_per_kg:,.2f} µmol/kg in surface box if fully dissolved")
print(f"Mass of surface box water: {M_water_surface_kg:,.2f} kg")
print(f"Mass of surface box water: {M_water_surface_Gt:,.6f} gigatonnes")

Lime addition: 742,500 tonnes Ca(OH)2
molar mass Ca(OH)2: 74.0920 g/mol
10021324839.388866
→ ΔTA1 ≈ 6.76 µmol/kg in surface box if fully dissolved
Mass of surface box water: 2,967,003,768,777,009.50 kg
Mass of surface box water: 2,967.003769 gigatonnes


# Model

In [18]:
# --- Initialize output ---
state = initial_state.copy()
states = [state.copy()]

pCO2_surf_values = []
pH_surf_values = []
Omega_calc_surf_values = []

pH_deep_values = []
Omega_calc_deep_values = []

PICPOC_dynamic_values = []
f_diss_dynamic_values = []
PIC_burial_values = []
POC_burial_values = []

J_air_values = []
J_air_mmol_m2_day_values = []


shock_time = 2.0  # year when the alkalinity shock occurs
if oae_mode == "oae":
    SHOCK_ADDED = True 
else:
    SHOCK_ADDED = False

ENABLE_CONVECTION = True  # False = disable density-driven mixing
                          # True  = enable density-driven mixing


for i, t in enumerate(times[1:], start=1):
    # current layer geometry (step index is i-1)
    h1_t      = float(h1[i-1])
    h2_t      = float(h2[i-1])
    delta_z_t = float(delta_z[i-1])
    scale_t   = h1_t / h2_t

    T1, T2, S1, S2, DIC1, DIC2, TA1, TA2 = state # units °C, psu, µmol/kg, µmol/kg
    p = params

    # Apply alkalinity shock once adjust DIC accordingly!!!
    if SHOCK_ADDED and abs(t - shock_time) < 1e-6:  # very small tolerance for float comparison
        TA1 += delta_TA1_umol_per_kg  # µmol/kg
        SHOCK_ADDED = False
        # NOTE: DIC1 is NOT changed here; atmospheric CO2 uptake via J_air will
        # gradually increase DIC1 after the shock.

    # Seasonal Forcing Terms
    seasonal_T1 = p['A_1T'] * np.cos(2 * np.pi * t + p['phi_1T']) # °C
    seasonal_T2 = p['A_2T'] * np.cos(2 * np.pi * t + p['phi_2T']) # °C
    seasonal_S1 = p['A_1S'] * np.cos(2 * np.pi * t + p['phi_1S']) # psu
    seasonal_S2 = p['A_2S'] * np.cos(2 * np.pi * t + p['phi_2S']) # psu

    # --- physical diffusion (mixing) for temperature and salinity ---
    mix_T1 =  KD_m2_yr * ((T2 - T1) / delta_z_t) / h1_t
    mix_T2 = -KD_m2_yr * ((T2 - T1) / delta_z_t) / h2_t
    mix_S1 =  KD_m2_yr * ((S2 - S1) / delta_z_t) / h1_t
    mix_S2 = -KD_m2_yr * ((S2 - S1) / delta_z_t) / h2_t

    # include in the total tendencies
    dT1 = (p['T1_star'] + seasonal_T1 - T1) / p['tau_1T'] + mix_T1
    dT2 = (p['T2_star'] + seasonal_T2 - T2) / p['tau_2'] + mix_T2
    dS1 = (p['S1_star'] + seasonal_S1 - S1) / p['tau_1S'] + mix_S1
    dS2 = (p['S2_star'] + seasonal_S2 - S2) / p['tau_2'] + mix_S2

    # Calculate pCO2surf using PyCO2SYS
    co2sys_surf = pyco2.sys(
        par1=TA1,
        par2=DIC1,
        par1_type=1,  # 1 = TA 
        par2_type=2,  # 2 = DIC
        salinity=S1,
        temperature=T1,
        pressure=0,  # sea surface pressure, in dbar

        total_silicate=0.0,     # explicitly zero
        total_phosphate=0.0,    # explicitly zero

        opt_k_carbonic=10,       # 10 = Lücker refit
        opt_k_bisulfate=1,      # 1 = Dickson (1990)
    )

    co2sys_deep = pyco2.sys(
        par1=TA2,
        par2=DIC2,
        par1_type=1,
        par2_type=2,
        salinity=S2,
        temperature=T2,
        pressure=h1_t + 0.5 * h2_t, # mid-depth of deep box, in dbar
        total_silicate=0.0,
        total_phosphate=0.0,
        opt_k_carbonic=10,
        opt_k_bisulfate=1,
    )

    # surface diagnostics
    pCO2_surf        = co2sys_surf["pCO2"]                 # µatm
    pH_surf          = co2sys_surf["pH"]
    Omega_calc_surf  = co2sys_surf["saturation_calcite"]   # unitless Ω_calcite

    # deep diagnostics
    pH_deep          = co2sys_deep["pH"]
    Omega_calc_deep  = co2sys_deep["saturation_calcite"]   # unitless Ω_calcite
    # Store diagnostics
    pH_surf_values.append(pH_surf)
    pH_deep_values.append(pH_deep)
    pCO2_surf_values.append(pCO2_surf)
    Omega_calc_surf_values.append(Omega_calc_surf)
    Omega_calc_deep_values.append(Omega_calc_deep)


    # ------------------- Air-Sea CO2 Flux Calculation -------------------
    # wind forcing: O(1) array lookup now
    U2_now = U2_for_step[i-1]  # m^2 s^-2 for this model step

    # -- Compute k (cm/hr -> m/s) --
    k_cm_hr = float(k_W14_from_U2(U2_now, T1))                      # cm hr^-1 (in-situ)
    k_m_yr = k_cm_hr * (0.01 * 24.0 * DAYS_PER_YEAR)                # m yr^-1 (in-situ)

    # -- CO2 solubility K0 (Weiss 1974) --
    K0_mol_kg_atm = K0_weiss74_CO2(T1, S1)                          # mol kg^-1 atm^-1

    # -- Bulk air-sea CO2 flux F = k * K0 * Δp -- 
    delta_p_atm = (pCO2_atm - pCO2_surf) * 1e-6                     # atm

    # J_air in mmol m^-2 day^-1 
    J_air_mmol_m2_day = (rho * k_m_yr * K0_mol_kg_atm * delta_p_atm) * 1000.0 / DAYS_PER_YEAR # mmol m^-2 day^-1
    J_air_mmol_m2_day_values.append(J_air_mmol_m2_day)
    
    # J_air in µmol kg^-1 yr^-1
    J_air = (k_m_yr * K0_mol_kg_atm * delta_p_atm) / h1_t * 1e6  # J_air in µmol C kg^-1 yr^-1 (air-sea CO2 flux expressed as C)                      
    J_air_values.append(J_air)

  
    # --- Dynamic PIC:POC rain ratio and CaCO3 dissolution fraction ---

    # --- dynamic PIC:POC rain ratio (surface calcification sensitivity) ---
    Omega_ratio_surf = np.clip(Omega_calc_surf / p['Omega_crit_calc'], 0.0, 1.0)
    PICPOC_dynamic = p['PICPOC_min'] + Omega_ratio_surf * (p['PICPOC_max'] - p['PICPOC_min'])
    PICPOC_dynamic_values.append(PICPOC_dynamic)

    # --- dynamic deep CaCO3 dissolution fraction (Ω-dependence) ---
    Omega_ratio_deep = 1 - np.clip(Omega_calc_deep/p['Omega_crit_diss'], 0.0, 1.0)
    f_diss_dynamic = p['f_diss_min'] + Omega_ratio_deep * (p['f_diss_max'] - p['f_diss_min'])
    f_diss_dynamic_values.append(f_diss_dynamic)

    # ------------------- Biogeochemical Tendencies -------------------
    # --- mixing (per mass) ---
    mix_DIC1 =  KD_m2_yr * ((DIC2 - DIC1) / delta_z_t) / h1_t
    mix_DIC2 = -KD_m2_yr * ((DIC2 - DIC1) / delta_z_t) / h2_t
    mix_TA1  =  KD_m2_yr * ((TA2  - TA1 ) / delta_z_t) / h1_t
    mix_TA2  = -KD_m2_yr * ((TA2  - TA1 ) / delta_z_t) / h2_t

    # --- biology (per mass) ---
    NCP = NCP_for_step[i-1]  

    bio_DIC1 = -(NCP +                 # organic C fixed
                    NCP * PICPOC_dynamic)    # inorganic C fixed as CaCO3
    
    bio_DIC2 =  (NCP * f_remin_POC +               # POC remin → DIC
                    NCP * PICPOC_dynamic * f_diss_dynamic) * scale_t  # PIC dissolution → DIC
    bio_TA1  = -2.0 * NCP * PICPOC_dynamic              # each mole CaCO3 formation lowers TA by 2
    bio_TA2  =  (2.0 * NCP * PICPOC_dynamic * f_diss_dynamic) * scale_t  # CaCO3 dissolution returns 2 TA

    POC_burial = NCP * (1.0 - f_remin_POC)  * scale_t
    POC_burial_values.append(POC_burial)
    PIC_burial = NCP * PICPOC_dynamic * (1.0 - f_diss_dynamic) * scale_t
    PIC_burial_values.append(PIC_burial)

    # -------------------------------------------------------------------------------------

    # freshwater dilution/concentration (µmol kg^-1 yr^-1)
    #dTA1_FW  = TA1 * (1.0 / S1) * dS1
    #dDIC1_FW = DIC1 * (1.0 / S1) * dS1
    #dTA2_FW  = TA2 * (1.0 / S2) * dS2
    #dDIC2_FW = DIC2 * (1.0 / S2) * dS2

    # final tendencies (all in µmol kg^-1 yr^-1)
    dDIC1 = J_air + mix_DIC1 + bio_DIC1 #+ dDIC1_FW
    dDIC2 =         mix_DIC2 + bio_DIC2 #+ dDIC2_FW
    dTA1  =         mix_TA1  + bio_TA1  #+ dTA1_FW
    dTA2  =         mix_TA2  + bio_TA2  #+ dTA2_FW

    # Tentative Euler Step
    T1_hat = T1 + dt * dT1 # °C
    T2_hat = T2 + dt * dT2 # °C
    S1_hat = S1 + dt * dS1 # psu
    S2_hat = S2 + dt * dS2 # psu
    DIC1_hat = DIC1 + dt * dDIC1 # µmol/kg
    DIC2_hat = DIC2 + dt * dDIC2 # µmol/kg
    TA1_hat = TA1 + dt * dTA1   # µmol/kg
    TA2_hat = TA2 + dt * dTA2  # µmol/kg


    # --- Potential Density Mixing Condition ---
    if ENABLE_CONVECTION:
        # Pressures at representative depths (surface box near 0; deep at its mid-depth)
        p1 = 0.0
        p2 = h1_t + 0.5 * h2_t  # meters ~ dbar

        # If rho1 > rho2: surface water is denser → convection occurs.
        # Else surface water is lighter → no convection.
        # Compute potential densities using TEOS-10
        rho1 = pot_density(S1_hat, T1_hat, p1, lon0, lat0)
        rho2 = pot_density(S2_hat, T2_hat, p2, lon0, lat0)

        # If surface water is denser → convective overturning
        if rho1 > rho2 - 1e-6:  # small tolerance to avoid numerical issues
            mix_T = (h1_t * T1_hat + h2_t * T2_hat) / (h1_t + h2_t) # °C
            mix_S = (h1_t * S1_hat + h2_t * S2_hat) / (h1_t + h2_t) # psu
            mix_DIC = (h1_t * DIC1_hat + h2_t * DIC2_hat) / (h1_t + h2_t) # µmol/kg
            mix_TA = (h1_t * TA1_hat + h2_t * TA2_hat) / (h1_t + h2_t)  # µmol/kg
            state = np.array([mix_T, mix_T, mix_S, mix_S, mix_DIC, mix_DIC, mix_TA, mix_TA])
        else:
            state = np.array([T1_hat, T2_hat, S1_hat, S2_hat, DIC1_hat, DIC2_hat, TA1_hat, TA2_hat])
    else:
        # convection disabled → always take predicted values
        state = np.array([T1_hat, T2_hat, S1_hat, S2_hat, DIC1_hat, DIC2_hat, TA1_hat, TA2_hat])

    states.append(state.copy())

# --- Save Results ---
columns = ['T1', 'T2', 'S1', 'S2', 'DIC1', 'DIC2', 'TA1', 'TA2']
df = pd.DataFrame(states, columns=columns)
df['time'] = times

# Add a starting dummy value to match length
pCO2_surf_values.insert(0, np.nan)
pH_surf_values.insert(0, np.nan)
Omega_calc_surf_values.insert(0, np.nan)

pH_deep_values.insert(0, np.nan)
Omega_calc_deep_values.insert(0, np.nan)

PICPOC_dynamic_values.insert(0, np.nan)
f_diss_dynamic_values.insert(0, np.nan)
PIC_burial_values.insert(0, np.nan)
POC_burial_values.insert(0, np.nan)

J_air_values.insert(0, np.nan)
J_air_mmol_m2_day_values.insert(0, np.nan)


# Add to dataframe
df['pCO2_surf'] = pCO2_surf_values
df['pH_surf'] = pH_surf_values
df['Omega_calc_surf'] = Omega_calc_surf_values

df['pH_deep'] = pH_deep_values
df['Omega_calc_deep'] = Omega_calc_deep_values

df['PICPOC_dynamic'] = PICPOC_dynamic_values
df['f_diss_dynamic'] = f_diss_dynamic_values
df['PIC_burial'] = PIC_burial_values
df['POC_burial'] = POC_burial_values

df['J_air'] = J_air_values
df['J_air_mmol_m2_day'] = J_air_mmol_m2_day_values


# Save to CSV
df.to_csv(DATA_PROCESSED / f"Model_{mld_mode}_{region_name}_{oae_mode}.csv", index=False)